## 1. Getting started: Kaggle's Titanic Competition

Kaggle is already established as the best place which hosts machine learning competitions. If you do not know it already, then it's time to do it.

__[Titanic Competition](https://www.kaggle.com/c/titanic)__ is perhaps the first competition which one should try. Of course, if you are already an experienced data scientist, than you can skip to an advanced competition.

The purpose of the competition is to learn if a passenger has survived or not. We illustrate some steps and ideas one can apply to compete in this learning competition using the available tools one can find in rapaio library.

In [62]:
%jars /home/ati/work/out/artifacts/rapaio_jar/rapaio.jar /home/ati/work/out/artifacts/rapaio_jar/fastutil-8.2.1.jar

In [102]:
import java.util.stream.*;

import rapaio.datasets.*;
import rapaio.data.*;
import rapaio.sys.*;
import rapaio.graphics.*;
import rapaio.graphics.plot.*;
import static rapaio.graphics.Plotter.*;
import rapaio.io.*;
import rapaio.core.distributions.*;
import rapaio.core.*;
import rapaio.core.tests.*;
import rapaio.core.tools.*;
import rapaio.core.stat.*;
import rapaio.ml.classifier.*;
import rapaio.ml.classifier.tree.*;
import rapaio.experiment.ml.eval.*;
import rapaio.ml.classifier.tree.*;
import rapaio.ml.classifier.ensemble.*;

In [64]:
WS.getPrinter().withGraphicWidth(800);
WS.getPrinter().withGraphicHeight(600);
WS.getPrinter().withTextWidth(100);

rapaio.printer.StandardPrinter@6bf79586

### 1.2 Get the data

The purpose of the competition is to predict which passengers have survived or not. The available data has two parts. The first part consists in a data set which contains what happened with some passengers and some related information like sex, cabin, age, class, etc. This data set contains information regarding their survival. The purpose why this data set contains survival data is because it will be used to train a model which learns how to decide if a passenger survives or not. This is the `train.csv`. The other file is a data set which contains data about another set of passenger, this time without knowing if they survived or not. They contain, however an identification number. This data set is `test.csv` and this is used to make predictions. Those predictions should be similar with the provided `gendermodel.csv`.

We also have to take a look of the data description provided on __[contest dedicated page](https://www.kaggle.com/c/titanic/data)__:

```
VARIABLE DESCRIPTIONS:
survival Survival
(0 = No; 1 = Yes)
pclass Passenger Class
(1 = 1st; 2 = 2nd; 3 = 3rd)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare
cabin Cabin
embarked Port of Embarkation
(C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored. The following are the definitions used
for sibsp and parch.

Sibling: Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse: Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent: Mother or Father of Passenger Aboard Titanic
Child: Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws. Some children travelled
only with a nanny, therefore parch=0 for them. As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.
```

The first step in our adventure is to download those 3 data file in *csv* format. You can do it from __[data section](https://www.kaggle.com/c/titanic/data)__ of the competition. Let's suppose you downloaded somewhere in a local folder. We will name this folder `data` folder, and actually it can have any name you would like.

### 1.2 Read train data from csv file

Because the data is small we can load the whole data in memory with no problems.

Let's see how we can load the data into memory. In rapaio the sets of data are loaded into the form of *frames* (`rapaio.data.Frame`). A frame is basically a tabular data, with columns for each variable (feature) and rows for each instance (in our case for each passenger).


A first try of loading the train data set and see what has happened is the following:


In [65]:
String root = "/home/ati/work/rapaio-kaggle/src/titanic/";
new Csv().read(root + "train.csv").printSummary();

Frame Summary
* rowCount: 891
* complete: 889/891
* varCount: 12
* varNames: 

 0. PassengerId : double  | 4.   Sex : nominal |  8.   Ticket : nominal |
 1.    Survived : binary  | 5.   Age : nominal |  9.     Fare : double  |
 2.      Pclass : double  | 6. SibSp : double  | 10.    Cabin : nominal |
 3.        Name : nominal | 7. Parch : double  | 11. Embarked : nominal |

      PassengerId         Survived           Pclass 
   Min. :   1.000     Min. : 0.000     Min. : 1.000 
1st Qu. : 223.500  1st Qu. : 0.000  1st Qu. : 2.000 
 Median : 446.000   Median : 0.000   Median : 3.000 
   Mean : 446.000     Mean : 0.384     Mean : 2.309 
2nd Qu. : 668.500  2nd Qu. : 1.000  2nd Qu. : 3.000 
   Max. : 891.000     Max. : 1.000     Max. : 3.000 
                                                    
                                                       Name           Sex            Age            SibSp 
                            "Braund, Mr. Owen Harris" :   1    male : 577          : 177     

How can we interpret the output of the frame's summary?

* We loaded a frame which has $891$ rows and $12$ columns (variables)
* From all the rows, $889$ are complete (non missing data)
* The name of the variables are listed, together with their types
* It follows a data summary for the frame: 6 number summary for numeric variables, most frequent levels for nominal variables

Let's inspect each variable and see how it fits our needs.

**PassengedId**

The type for this variable is index (integer values). This field looks like an identifier for the passenger, so from our point of view the sorting is not required. What we can do, but is not required, is to change the field type to nominal. Anyway, we do not need this field for learning since it should be unique for each instance, thus the predictive power is null. We will ignore it for now since we will not consider it for learning

**Survived**

This is our target variable. It is parsed as binary, but since we do classification, we will change it's type to nominal. We do that directly from the csv parsing, by indicating that we want Survived parsed as nominal variable:


In [66]:
new Csv()
.withTypes(VType.NOMINAL, "Survived")
.read(root + "train.csv")
.printSummary();

Frame Summary
* rowCount: 891
* complete: 889/891
* varCount: 12
* varNames: 

 0. PassengerId : double  | 4.   Sex : nominal |  8.   Ticket : nominal |
 1.    Survived : nominal | 5.   Age : nominal |  9.     Fare : double  |
 2.      Pclass : double  | 6. SibSp : double  | 10.    Cabin : nominal |
 3.        Name : nominal | 7. Parch : double  | 11. Embarked : nominal |

      PassengerId  Survived           Pclass                                                         Name 
   Min. :   1.000   0 : 549     Min. : 1.000                              "Braund, Mr. Owen Harris" :   1 
1st Qu. : 223.500   1 : 342  1st Qu. : 2.000  "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" :   1 
 Median : 446.000             Median : 3.000                               "Heikkinen, Miss. Laina" :   1 
   Mean : 446.000               Mean : 2.309         "Futrelle, Mrs. Jacques Heath (Lily May Peel)" :   1 
2nd Qu. : 668.500            2nd Qu. : 3.000                             "Allen, Mr. Will

And notice how type of the `Survived` variable changed to nominal.


**Pclass**

This variable has index type. We can keep it like it is or we can change it to nominal. Both ways can be useful. For example parsed as index could give an interpretation to the order. We can say that somehow, because of ordering class 1 is lower than class 2, and class 2 is between classes 1 and 3. At the same time we can keep it as nominal if we do not want to use the ordering. Let's choose nominal for now, considering that 1,2 and 3 are just labels for type of tickets, with no other meaning attached. We proceed in the same way:

In [67]:
new Csv()
.withTypes(VType.NOMINAL, "Survived", "Pclass")
.read(root + "train.csv")
.printSummary();

Frame Summary
* rowCount: 891
* complete: 889/891
* varCount: 12
* varNames: 

 0. PassengerId : double  | 4.   Sex : nominal |  8.   Ticket : nominal |
 1.    Survived : nominal | 5.   Age : nominal |  9.     Fare : double  |
 2.      Pclass : nominal | 6. SibSp : double  | 10.    Cabin : nominal |
 3.        Name : nominal | 7. Parch : double  | 11. Embarked : nominal |

      PassengerId  Survived   Pclass                                                         Name 
   Min. :   1.000   0 : 549  3 : 491                              "Braund, Mr. Owen Harris" :   1 
1st Qu. : 223.500   1 : 342  1 : 216  "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" :   1 
 Median : 446.000            2 : 184                               "Heikkinen, Miss. Laina" :   1 
   Mean : 446.000                            "Futrelle, Mrs. Jacques Heath (Lily May Peel)" :   1 
2nd Qu. : 668.500                                                "Allen, Mr. William Henry" :   1 
   Max. : 891.000             

Notice that we append the variable name after `Survived`. This is possible since the `withTypes` method specify a type, and follows a dynamic array of strings, for the names of variables.

**Name**

This is the passenger names and the values are unique. As it is, the predictive power of this field is null. We keep it as it is. Note that it contains valuable information, but not in this direct form.

**Sex**

This field specifies the gender of the passenger. We have $577$ males and $314$ females.

**Age**

This field specifies the age of an passenger. We would expect that to parse this variable as numeric or at leas index, but is nominal. Why that happened? Notice that the values looks like numbers. But the first value (the most frequent one, $117$ instances) has nothing specified. Well, the variable is nominal has to do with how *Csv* parsing handles missing values. By default, the *csv* parsing considers as missing values only the string "?". But the most frequent value in this field is empty string "". This means that empty string is not considered a missing value. Because empty string can't produce numbers from parsing, the variable is *promoted* to nominal.

We can customize the missing value handling by specifying the valid strings for that purpose. We use `.useNAValues(String...naValues)` to tell the parser all the valid strings which are missing values. In our case we want just the empty string to be a missing value. When the parser will found an empty string it will set the variable value as missing value. It will *not promote* variable to nominal, since a missing value is a legal value.

In [68]:
new Csv()
.withNAValues("")
.withTypes(VType.NOMINAL, "Survived", "Pclass")
.read(root + "train.csv")
.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : double  | 4.   Sex : nominal |  8.   Ticket : nominal |
 1.    Survived : nominal | 5.   Age : double  |  9.     Fare : double  |
 2.      Pclass : nominal | 6. SibSp : double  | 10.    Cabin : nominal |
 3.        Name : nominal | 7. Parch : double  | 11. Embarked : nominal |

      PassengerId  Survived   Pclass                                                         Name 
   Min. :   1.000   0 : 549  3 : 491                              "Braund, Mr. Owen Harris" :   1 
1st Qu. : 223.500   1 : 342  1 : 216  "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" :   1 
 Median : 446.000            2 : 184                               "Heikkinen, Miss. Laina" :   1 
   Mean : 446.000                            "Futrelle, Mrs. Jacques Heath (Lily May Peel)" :   1 
2nd Qu. : 668.500                                                "Allen, Mr. William Henry" :   1 
   Max. : 891.000             

Notice what happened: *Age* field is now numeric and it contains $177$ missing values.

**SibSp**

It's meaning is "siblings/spouses". It's parsed as index, which is natural. In pathological cases with sick imagination we can consider a "quarter of a wife" for example.

**Parch**

It's meaning is "parents/children". It is naturally parsed as index.

**Ticket**

This is the code of the ticket. Probably a family can have the same ticket, thus must be the reason why the frequencies have values up to $$7$$. This field is nominal. It has low predictive power used directly. Perhaps contains valuable information, but used directly in row format would not help much.

**Fare**

This is the price for passenger fare and should be numeric, like it is.

**Cabin**

Code of the passenger's cabin, parsed as nominal. Same notes as for `Ticket` variable.

**Embarked**

Code for the embarking city, which could be: C = Cherbourg, Q = Queenstown, S = Southampton. It's parsed as nominal and has $2$ missing values.

If we are content with our parsing, we load data into a data frame for later use:

In [69]:
Frame train = new Csv().withNAValues("").withTypes(VType.NOMINAL, "Survived", "Pclass")
.read(root + "train.csv");
train.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : double  | 4.   Sex : nominal |  8.   Ticket : nominal |
 1.    Survived : nominal | 5.   Age : double  |  9.     Fare : double  |
 2.      Pclass : nominal | 6. SibSp : double  | 10.    Cabin : nominal |
 3.        Name : nominal | 7. Parch : double  | 11. Embarked : nominal |

      PassengerId  Survived   Pclass                                                         Name 
   Min. :   1.000   0 : 549  3 : 491                              "Braund, Mr. Owen Harris" :   1 
1st Qu. : 223.500   1 : 342  1 : 216  "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" :   1 
 Median : 446.000            2 : 184                               "Heikkinen, Miss. Laina" :   1 
   Mean : 446.000                            "Futrelle, Mrs. Jacques Heath (Lily May Peel)" :   1 
2nd Qu. : 668.500                                                "Allen, Mr. William Henry" :   1 
   Max. : 891.000             

### 1.3 Read test data from *csv* file

Once we have a training frame we can load also the test data. We do that to take a look at the frame and because data is small and there is no memory or time problem cost associated with it. To avoid adding again the *csv* options and to get identical levels nominal variables, we use a different way to parse the data set. We specify variable types by frame templates:

In [70]:
Frame test = new Csv().withNAValues("").withTemplate(train).read(root + "test.csv");

Instead to specify again the preferred types for variables, we use train frame as a template for variable types. This has also the side effect that the encoding of categorical variables is identical.

In [71]:
test.printSummary();

Frame Summary
* rowCount: 418
* complete: 87/418
* varCount: 11
* varNames: 

 0. PassengerId : double  | 4.    Age : double  |  8.     Fare : double  |
 1.      Pclass : nominal | 5.  SibSp : double  |  9.    Cabin : nominal |
 2.        Name : nominal | 6.  Parch : double  | 10. Embarked : nominal |
 3.         Sex : nominal | 7. Ticket : nominal |                         

       PassengerId   Pclass                                                  Name           Sex 
   Min. :  892.000  3 : 218                          "Connolly, Miss. Kate" :   1    male : 266 
1st Qu. :  996.250  1 : 107                              "Kelly, Mr. James" :   1  female : 152 
 Median : 1100.500  2 :  93              "Wilkes, Mrs. James (Ellen Needs)" :   1               
   Mean : 1100.500                              "Myles, Mr. Thomas Francis" :   1               
2nd Qu. : 1204.750                                       "Wirz, Mr. Albert" :   1               
   Max. : 1309.000           "Hirvonen,

We can note that we don't have *Survived* variable anymore. This is correct since this is what we have to predict. Note also that the types for the remaining variables are the same with training data set.

## 2. Build simple models

### 2.1 Build a majority model

To make a first submission we will build a very simple model, which classifies with a single value all instances. This value is the majority label.

Let's inspect at how target variable look like.

In [72]:
DVector.fromCount(false, train.rvar("Survived")).printSummary();

    0   1
    -   -
  549 342


As we already new from the summary, the number of passengers who didn't survived is lower than those who did. Let's see percentages:

In [73]:
DVector.fromCount(false, train.rvar("Survived")).normalize().printSummary();

          0         1
          -         -
  0.6161616 0.3838384


We note that there are about $61\%$ of passengers who did not survived. We will create a submit data set, which we will save for later submission. How we can do that?

In [74]:
VarNominal prediction = VarNominal.from(test.rowCount(), row -> "0").withName("Survived");
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"), prediction);

new Csv().withQuotes(false).write(submit, root + "majority_submit.csv");

In the first line we created a new nominal variable. The size of the new variable is the number of rows from the test frame. For each row we produce the same label `"0"`. We name this variable `Survived`.

In the second line we created a new frame taking the variable named `PassengerId` from the test data set and the new prediction variable.

In the last line we wrote a new csv file with the csv parsing utility, taking care to not write quotes. We can submit this file and see which are the results.

![Submission result with majority classifier](images/titanic-majority-submit.png)

### 2.2 Build a simple gender model

It has been said that "women and children first" really happened during Titanic tragedy. If this was true or not, we do not know. But we can use data to see if we are hearing the same story. For now we will take the gender and see if it had an influence. We will build a contingency table for variables `Sex` and `Survived`.

In [75]:
DTable.fromCounts(train.rvar("Sex"), train.rvar("Survived"), false).printSummary();

          0   1 total
   male 468 109   577
 female  81 233   314
  total 549 342   891


On rows we have levels of `Sex` variable. On columns we have levels of `Sex` variable. Cells are computed as counts. What we see is that there are a lot of men who did not survived and a lot of women who does. We will normalize on rows to take a closer look.

In [76]:
DTable.fromCounts(train.rvar("Sex"), train.rvar("Survived"), false).normalizeOnRows().printSummary();

                0         1 total
   male 0.8110919 0.1889081     1
 female 0.2579618 0.7420382     1
  total 1.0690536 0.9309464     2


It seems that men survived with a rate of $0.19$ and women with $0.74$. The values are so obvious, we need no hypothesis testing to check that this variable is significant for classification. We will build a simple model where we predict as survived all the women and not survived all the men.

In [77]:
Var prediction = VarNominal.from(test.rowCount(), row -> test.getLabel(row, "Sex").equals("male") 
? "0" : "1").withName("Survived");
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"), prediction);
new Csv().withQuotes(false).write(submit, root + "gender_submit.csv");

__![Submission result with gender classifier](images/titanic-gender-submit.png)__

## 3. Tree model

Building models in the manual way is often not the way to go. This process is tedious and time consuming. There are already built automated procedures, which incorporate miscellaneous approaches to learn a classifier. One of the often used models is the decision tree. Decision trees are greedy local approximations build in a recursive greedy fashion. Often the split decision at node level uses a single feature. At leave nodes a simple majority classifier creates the classification rule.

### 3.1 Gender model with decision tree
Initially we will build a CART decision tree using as input feature the *Sex* variable. We do this to exemplify how a manual rule can be created in an automated fashion.

In [78]:
Frame tr = train.mapVars("Survived,Sex");
CTree tree = CTree.newCART();
tree.fit(tr, "Survived");
tree.printSummary();

CTree model

Description:
CTree {varSelector=VarSelector[ALL];
minCount=1;
maxDepth=-1;
tests=BINARY:BinaryBinary,DOUBLE:NumericBinary,NOMINAL:NominalBinary,INT:NumericBinary;
func=GiniGain;
split=ToAllWeighted;
}

Capabilities:
types inputs/targets: BINARY,INT,NOMINAL,DOUBLE/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:
input vars: 
 0. Sex : NOMINAL  |               

target vars:
> Survived : NOMINAL [?,0,1]


total number of nodes: 3
total number of leaves: 2
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.139648]
|   |- 1. Sex = 'female'    314/81 1 (0.258 0.742 ) *
|   |- 2. Sex != 'female'    577/109 0 (0.811 0.189 ) *


Taking a closer look at the last three rows from the output, one can identify our manual rule. Basically the interpretation is: *"all the females survived, all the males did not"*. For exemplification purposes we build also the submit file.

In [79]:
// fit the tree to test data frame
CPrediction pred = tree.predict(test);
// build teh submission
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"),pred.firstClasses().withName("Survived"));
// write to a submit file
new Csv().withQuotes(false).write(submit, root + "tree1-model.csv");

### 3.2 Enrich tree by using other features

Our training data set has more than a single input feature. Thus We can state we didn't use all the information available. We will add now the class and embarking port and see how it behaves.

In [96]:
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked");

CTree tree = CTree.newCART();
tree.fit(tr, "Survived");
tree.printSummary();

CPrediction pred = tree.predict(test);
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"),pred.firstClasses().withName("Survived"));
new Csv().withQuotes(false).write(submit, root + "tree2-model.csv");

CTree model

Description:
CTree {varSelector=VarSelector[ALL];
minCount=1;
maxDepth=-1;
tests=BINARY:BinaryBinary,DOUBLE:NumericBinary,NOMINAL:NominalBinary,INT:NumericBinary;
func=GiniGain;
split=ToAllWeighted;
}

Capabilities:
types inputs/targets: BINARY,INT,NOMINAL,DOUBLE/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:
input vars: 
 0. Sex : NOMINAL  | 1. Pclass : NOMINAL  | 2. Embarked : NOMINAL  |

target vars:
> Survived : NOMINAL [?,0,1]


total number of nodes: 17
total number of leaves: 9
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.139648]
|   |- 1. Sex = 'female'    314/81 1 (0.258 0.742 ) [0.0204665]
|   |   |- 3. Pclass = '2'    76/6 1 (0.079 0.921 ) [0.0003369]
|   |   |   |- 7. Embarked = 'Q'    2/0 1 (0 1 ) *
|   |   |   |- 8. Embarked != 'Q'    74/6 1 (0.081 0.919 ) [0.0013737]
|   |   |   |   |- 13. Embarked = 'C'    7/0 1 (0 1 ) *
|   |  

The tree is much richer and there are more chances to be better. This is what happened after submission.

__![Results after submission of enriched tree](images/titanic-tree2-submit.png)__

**Nice!**. We advanced $704$ positions and improved our score with $0.01435$. On public leader board we have a nice $0.77990$ accuracy score.

### 3.3 Overfitting with trees

What about using other input features to improve our prediction accuracy? There are some of them which we can include directly, with no changes: *Age*,*Fare*,*SibSp* and *Parch*.

We can change the script slightly, to include those new input features. But we can do better, we can use cross-validation to estimate what will happen.

In [97]:
CTree tree = CTree.newCART();
CEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked"),"Survived", tree, 10);
CEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"),"Survived", tree, 10);


CrossValidation with 10 folds
CV  1:  acc=0.755556, mean=0.755556, se=NaN
CV  2:  acc=0.696629, mean=0.726092, se=0.041667
CV  3:  acc=0.842697, mean=0.764960, se=0.073486
CV  4:  acc=0.820225, mean=0.778777, se=0.066058
CV  5:  acc=0.797753, mean=0.782572, se=0.057834
CV  6:  acc=0.775281, mean=0.781357, se=0.051814
CV  7:  acc=0.842697, mean=0.790119, se=0.052676
CV  8:  acc=0.786517, mean=0.789669, se=0.048785
CV  9:  acc=0.752809, mean=0.785574, se=0.047259
CV 10:  acc=0.797753, mean=0.786792, se=0.044723
Mean accuracy:0.786792
SE: 0.044723     (Standard error)

CrossValidation with 10 folds
CV  1:  acc=0.777778, mean=0.777778, se=NaN
CV  2:  acc=0.842697, mean=0.810237, se=0.045905
CV  3:  acc=0.741573, mean=0.787349, se=0.051237
CV  4:  acc=0.797753, mean=0.789950, se=0.042157
CV  5:  acc=0.808989, mean=0.793758, se=0.037489
CV  6:  acc=0.775281, mean=0.790678, se=0.034369
CV  7:  acc=0.808989, mean=0.793294, se=0.032128
CV  8:  acc=0.820225, mean=0.796660, se=0.031232
CV  9:  a

0.7923845193508114

Notice that the 10-crossfold estimator of the accuracy has dropped with a large quantity. What happens? We can have an idea if we take a look at the learned tree:

In [98]:
tree.fit(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"), "Survived");
tree.printSummary();

CTree model

Description:
CTree {varSelector=VarSelector[ALL];
minCount=1;
maxDepth=-1;
tests=BINARY:BinaryBinary,DOUBLE:NumericBinary,NOMINAL:NominalBinary,INT:NumericBinary;
func=GiniGain;
split=ToAllWeighted;
}

Capabilities:
types inputs/targets: BINARY,INT,NOMINAL,DOUBLE/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:
input vars: 
 0.      Sex : NOMINAL  | 3.   Age : DOUBLE  | 6. Parch : DOUBLE  |
 1.   Pclass : NOMINAL  | 4.  Fare : DOUBLE  |                     
 2. Embarked : NOMINAL  | 5. SibSp : DOUBLE  |                     

target vars:
> Survived : NOMINAL [?,0,1]


total number of nodes: 511
total number of leaves: 256
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.5731486]
|   |- 1. Age <= 37.5    703/271 0 (0.608 0.392 ) [0.6304902]
|   |   |- 3. Age <= 27.5    514/193 0 (0.606 0.394 ) [0.11952]
|   |   |   |- 7. Sex = 'female'    186/55 1 (0.

|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |- 470. Fare > 25.9625    8/3 0 (0.615 0.385 ) [0]
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |- 483. Fare <= 26.275    6/2 0 (0.635 0.365 ) *
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |- 484. Fare > 26.275    2/1 1 (0.5 0.5 ) *
|   |   |   |   |   |   |   |   |   |   |   |- 338. Fare > 27.1354    2/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |   |- 284. Fare > 28.3604    39/18 1 (0.436 0.564 ) [0.0390194]
|   |   |   |   |   |   |   |   |   |   |   |- 339. Fare <= 30.5979    6/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |   |   |   |- 340. Fare > 30.5979    33/15 0 (0.507 0.493 ) [0.0374853]
|   |   |   |   |   |   |   |   |   |   |   |   |- 385. Fare <= 35.25    4/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |   |   |   |- 386. Fare > 35.25    29/14 1 (0.459 0.541 ) [0.0162161]
|   |   |   |   |   |   |   |   |   |   |   |   |   |- 423. Fare <= 80.52915    22/10 0 (0.527 0

|   |   |   |   |   |   |   |   |   |   |- 290. Fare > 19.125    11/2 1 (0.157 0.843 ) [0.0162595]
|   |   |   |   |   |   |   |   |   |   |   |- 345. Fare <= 21.0375    4/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |   |   |   |- 346. Fare > 21.0375    7/2 1 (0.312 0.688 ) [0]
|   |   |   |   |   |   |   |   |   |   |   |   |- 389. Fare <= 21.71665    1/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |   |   |   |- 390. Fare > 21.71665    6/1 1 (0.087 0.913 ) [0.0149778]
|   |   |   |   |   |   |   |   |   |   |   |   |   |- 427. Fare <= 23.35    4/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |   |   |   |   |   |- 428. Fare > 23.35    2/1 1 (0.209 0.791 ) [0]
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |- 455. Fare <= 23.725    1/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |- 456. Fare > 23.725    1/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |   |- 232. Fare > 24.075    5/1 0 (0.871 0.129 ) [0]
|   |   |   |   |   |   |   |   |   |   |- 291. F

|   |   |   |   |   |   |   |   |   |   |   |- 353. Fare <= 54.7979    1/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |   |   |- 354. Fare > 54.7979    6/2 1 (0.333 0.667 ) *
|   |   |   |   |   |   |   |   |   |- 240. Fare > 56.7479    1/0 1 (0 1 ) *
|   |   |   |   |   |   |   |- 134. Fare > 61.8    8/0 0 (1 0 ) *
|   |   |   |   |   |   |- 84. Fare > 101.0854    5/2 1 (0.15 0.85 ) [0.0177907]
|   |   |   |   |   |   |   |- 135. Fare <= 369.9271    3/2 1 (0.346 0.654 ) [0]
|   |   |   |   |   |   |   |   |- 183. Fare <= 170.8896    1/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |- 184. Fare > 170.8896    2/0 0 (1 0 ) *
|   |   |   |   |   |   |   |- 136. Fare > 369.9271    2/0 1 (0 1 ) *
|   |- 2. Age > 37.5    365/123 0 (0.639 0.361 ) [0.1561569]
|   |   |- 5. Sex = 'female'    118/32 1 (0.247 0.753 ) [0.0575632]
|   |   |   |- 11. Fare <= 49.1896    76/29 1 (0.408 0.592 ) [0.0579414]
|   |   |   |   |- 23. Pclass = '2'    18/3 1 (0.182 0.818 ) [0.0102224]
|   |   |   |   |   |-

|   |   |   |   |   |   |   |   |   |   |- 306. Fare > 13.93125    9/0 0 (1 0 ) *
|   |   |   |   |   |   |   |   |   |- 252. Fare > 15.1729    2/0 1 (0 1 ) *
|   |   |   |   |   |   |   |   |- 194. Fare > 15.3729    9/0 0 (1 0 ) *
|   |   |   |   |   |   |   |- 148. Fare > 22.4646    1/0 1 (0 1 ) *
|   |   |   |   |   |   |- 96. Fare > 23.35    15/0 0 (1 0 ) *
|   |   |   |- 14. Fare > 26.26875    80/23 0 (0.714 0.286 ) [0.0054363]
|   |   |   |   |- 29. Pclass = '2'    2/0 0 (1 0 ) *
|   |   |   |   |- 30. Pclass != '2'    78/23 0 (0.704 0.296 ) [0]
|   |   |   |   |   |- 55. SibSp <= 0.5    54/15 0 (0.749 0.251 ) [0.0348365]
|   |   |   |   |   |   |- 97. Fare <= 31.6604    27/11 0 (0.605 0.395 ) [0]
|   |   |   |   |   |   |   |- 149. Fare <= 26.41875    1/0 1 (0 1 ) *
|   |   |   |   |   |   |   |- 150. Fare > 26.41875    26/10 0 (0.636 0.364 ) [0.028739]
|   |   |   |   |   |   |   |   |- 195. Fare <= 29.85    17/5 0 (0.735 0.265 ) [0]
|   |   |   |   |   |   |   |   |   |- 253. 


Notice how large is the tree. Basically the tree was full grown and overfit the training data set too much. We can ask ourselves why that happens? Why it happens now, and did not happened when we had fewer inputs? The answer is that it happened also before. But it's consequences were not so drastic.

The first tree used for training just $3$ input nominal features. Notice that all three features are nominal. The maximum number of groups which one can form is given by the product of the number of levels for each feature. This total maximal number is $2*3*3=18$. It practically exhausted the discrimination potential of those features. It did overfit in that reduced space of features. When we apply the model to the whole data set, the effect of exhaustion is not seen anymore.

The second tree does the same thing, but this time in a richer space, with added input dimensions. Compared with the full feature space, we see the effect.

There are two approaches to avoid overfit for a decision tree. The first approach is to stop learning up to the moment when we exhaust the data. The name for this approach is *early stop*. We can do that by specifying some parameters of the tree model:

* Set a minimum number of instances for leaf node
* Set a maximal depth for the tree
* Not implemented yet, but easy to do: complexity threshold, maximal number of nodes in a tree

The second approach is to prune the tree. Pruning procedure consists of growing the full tree and later on removing some nodes if they do not provide some type of gain. Currently we implemented only *reduced error pruning strategy*.

We will test with 10-fold cross validation an early-stopping strategy to see how it works.

In [99]:
CEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"),
"Survived", tree.withMaxDepth(12).withMinCount(4), 10);


CrossValidation with 10 folds
CV  1:  acc=0.800000, mean=0.800000, se=NaN
CV  2:  acc=0.741573, mean=0.770787, se=0.041314
CV  3:  acc=0.775281, mean=0.772285, se=0.029328
CV  4:  acc=0.797753, mean=0.778652, se=0.027122
CV  5:  acc=0.786517, mean=0.780225, se=0.023750
CV  6:  acc=0.786517, mean=0.781273, se=0.021398
CV  7:  acc=0.775281, mean=0.780417, se=0.019664
CV  8:  acc=0.842697, mean=0.788202, se=0.028571
CV  9:  acc=0.831461, mean=0.793009, se=0.030367
CV 10:  acc=0.808989, mean=0.794607, se=0.029073
Mean accuracy:0.794607
SE: 0.029073     (Standard error)


0.7946067415730337


I tried some values, just to show that we can do something about it, but the progress did not appear. We should try a different approach, and that is an ensemble. Next session contains directions on how to build such an ensemble.

## 4. CForest model

Random forests are well-known to work well when the irreducible error from the training data is high. This is probably the case of this Titanic data set. We have reasons to believe that this is the situation since it was a tragedy. A lot of random or not-so-expected things happened. That happened despite of the bravery and the sacrifice of the crew and others.

Random forests are the invention of [Leo Breiman](https://en.wikipedia.org/wiki/Leo_Breiman). The first design was a joint effort together with [Adele Cutler](http://www.math.usu.edu/adele/). The base of random forests is bagging (or **b**ootstrapp **ag**gregation). On top of that, selecting just a random limited number of variables at each node is the core of the algorithm.

We will work with random forests for now. This ensemble is mode robust and is capable of obtaining much better results than a single tree. At the same time we will introduce 10-fold cross validation to check our progress and estimate the error produced.

In the beginning we will use 10-fold cross validation for estimating the accuracy on public leader board.

We will test first with 10 fold cv the tree model.

In [101]:
CEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked"), "Survived", CTree.newCART(), 10);


CrossValidation with 10 folds
CV  1:  acc=0.666667, mean=0.666667, se=NaN
CV  2:  acc=0.775281, mean=0.720974, se=0.076802
CV  3:  acc=0.842697, mean=0.761548, se=0.088815
CV  4:  acc=0.775281, mean=0.764981, se=0.072841
CV  5:  acc=0.775281, mean=0.767041, se=0.063250
CV  6:  acc=0.808989, mean=0.774032, se=0.059108
CV  7:  acc=0.820225, mean=0.780631, se=0.056712
CV  8:  acc=0.865169, mean=0.791199, se=0.060416
CV  9:  acc=0.775281, mean=0.789430, se=0.056763
CV 10:  acc=0.764045, mean=0.786891, se=0.054115
Mean accuracy:0.786891
SE: 0.054115     (Standard error)


0.7868913857677903

### 4.1 Our first random forest

The name of the random forest implementation is `CForest`. To build a new ensemble of trees, one have to instantiate it in the following way:

In [86]:
Classifier rf = CForest.newRF();

There are a lot of things which can be customized for a random forest. Among them one can change:

* Number of trees for classification
* Which kind of weak classifier to use (you can customize this customized accordingly, like any other classifier)
* Number of threads in pool (if you want to use parallelism)
* What to do after each running step

Let's build one and use ore new cross validation procedure to estimate it's error.

In [87]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked");
CForest rf = CForest.newRF().withRuns(100);
CEvaluation.cv(tr, "Survived", rf, 10);


CrossValidation with 10 folds
CV  1:  acc=0.733333, mean=0.733333, se=NaN
CV  2:  acc=0.808989, mean=0.771161, se=0.053496
CV  3:  acc=0.842697, mean=0.795006, se=0.056006
CV  4:  acc=0.764045, mean=0.787266, se=0.048278
CV  5:  acc=0.842697, mean=0.798352, se=0.048607
CV  6:  acc=0.831461, mean=0.803870, se=0.045528
CV  7:  acc=0.730337, mean=0.793365, se=0.049998
CV  8:  acc=0.820225, mean=0.796723, se=0.047253
CV  9:  acc=0.820225, mean=0.799334, se=0.044890
CV 10:  acc=0.674157, mean=0.786816, se=0.057949
Mean accuracy:0.786816
SE: 0.057949     (Standard error)


0.7868164794007491

Well, an identical output. This is due to the fact that our variables are already exhausted by the tree. It looks like an underfit. If one consider bias variance trade off, one can see this as high bias. We need to enrich our feature space to improve our performance.

Let's be direct and test what would happen if we would use all our directly usable features? This time we will fit also the training data set, to see the distribution of the training error.

In [90]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newRF().withRuns(100);
CEvaluation.cv(tr, "Survived", rf, 10);

rf.fit(tr, "Survived");
CPrediction fit = rf.predict(test);
new Confusion(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();


CrossValidation with 10 folds
CV  1:  acc=0.822222, mean=0.822222, se=NaN
CV  2:  acc=0.853933, mean=0.838077, se=0.022423
CV  3:  acc=0.876404, mean=0.850853, se=0.027222
CV  4:  acc=0.842697, mean=0.848814, se=0.022598
CV  5:  acc=0.775281, mean=0.834107, se=0.038268
CV  6:  acc=0.842697, mean=0.835539, se=0.034407
CV  7:  acc=0.764045, mean=0.825325, se=0.041433
CV  8:  acc=0.831461, mean=0.826092, se=0.038421
CV  9:  acc=0.842697, mean=0.827937, se=0.036363
CV 10:  acc=0.719101, mean=0.817054, se=0.048579
Mean accuracy:0.817054
SE: 0.048579     (Standard error)
> Confusion

 Ac\Pr |    0    1 | total
 ----- |    -    - | -----
     0 | >527   22 |   549
     1 |   46 >296 |   342
 ----- |    -    - | -----
 total |  573  318 |   891


Complete cases 891 from 891
Acc: 0.9236813         (Accuracy )
F1:  0.9393939         (F1 score / F-measure)
MCC: 0.8378872         (Matthew correlation coefficient)
Pre: 0.9197208         (Precision)
Rec: 0.9599271         (Recall)
G:   0.9396089   

This time we have a good example of overfit. Why is that? Look at the confusion matrix on the training set. We fit too well the training data. This data set is well known for its high irreducible error. And there is an explanation for that. During the tragic event a lot of exceptional things happened. For example I read somewhere that an old lady which had a dog was not allowed to embark with her pet due to regulations. As a consequence she decided to not leave it and she chose to die with him. It's close to impossible to learn those kind of things, even if the information would be available. 

We should reduce the error somehow. We can try to decrease the overfit by adding more learners. Let's see if that would be enough for our purpose.

In [93]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newRF().withRuns(500);
CEvaluation.cv(tr, "Survived", rf, 10);

rf.fit(tr, "Survived");
CPrediction fit = rf.predict(test);
new Confusion(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();


CrossValidation with 10 folds
CV  1:  acc=0.822222, mean=0.822222, se=NaN
CV  2:  acc=0.831461, mean=0.826841, se=0.006533
CV  3:  acc=0.876404, mean=0.843362, se=0.028986
CV  4:  acc=0.820225, mean=0.837578, se=0.026343
CV  5:  acc=0.786517, mean=0.827366, se=0.032279
CV  6:  acc=0.831461, mean=0.828048, se=0.028919
CV  7:  acc=0.752809, mean=0.817300, se=0.038803
CV  8:  acc=0.831461, mean=0.819070, se=0.036271
CV  9:  acc=0.842697, mean=0.821695, se=0.034831
CV 10:  acc=0.719101, mean=0.811436, se=0.046162
Mean accuracy:0.811436
SE: 0.046162     (Standard error)
> Confusion

 Ac\Pr |    0    1 | total
 ----- |    -    - | -----
     0 | >526   23 |   549
     1 |   44 >298 |   342
 ----- |    -    - | -----
 total |  570  321 |   891


Complete cases 891 from 891
Acc: 0.9248036         (Accuracy )
F1:  0.9401251         (F1 score / F-measure)
MCC: 0.8402332         (Matthew correlation coefficient)
Pre: 0.922807         (Precision)
Rec: 0.9581056         (Recall)
G:   0.9402907    

This is slightly better than before. But the difference does not look significantly better than previous. We will use a simple pre-pruning strategy is to limit the number instances in leaf nodes. We set the minimum count to $3$.

In [95]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newRF()
.withClassifier(CTree.newCART().withMinCount(3))
.withRuns(100);
CEvaluation.cv(tr, "Survived", rf, 10);

rf.fit(tr, "Survived");
CPrediction fit = rf.predict(test);
new Confusion(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();


CrossValidation with 10 folds
CV  1:  acc=0.800000, mean=0.800000, se=NaN
CV  2:  acc=0.820225, mean=0.810112, se=0.014301
CV  3:  acc=0.887640, mean=0.835955, se=0.045889
CV  4:  acc=0.820225, mean=0.832022, se=0.038285
CV  5:  acc=0.775281, mean=0.820674, se=0.041752
CV  6:  acc=0.853933, mean=0.826217, se=0.039736
CV  7:  acc=0.775281, mean=0.818941, se=0.041066
CV  8:  acc=0.808989, mean=0.817697, se=0.038182
CV  9:  acc=0.853933, mean=0.821723, se=0.037703
CV 10:  acc=0.719101, mean=0.811461, se=0.048132
Mean accuracy:0.811461
SE: 0.048132     (Standard error)
> Confusion

 Ac\Pr |    0    1 | total
 ----- |    -    - | -----
     0 | >520   29 |   549
     1 |   67 >275 |   342
 ----- |    -    - | -----
 total |  587  304 |   891


Complete cases 891 from 891
Acc: 0.8922559         (Accuracy )
F1:  0.915493         (F1 score / F-measure)
MCC: 0.7706188         (Matthew correlation coefficient)
Pre: 0.8858603         (Precision)
Rec: 0.9471767         (Recall)
G:   0.9160056    

Notice that we changed the classifier used by `CForest`. This is the same classifier used by default by random forest. We do this because we customized the classifier by changing the min count parameter.

That had indeed some effect. However after submitting to competition we did not saw any improvement. We should look forward to engineer a little bit our features for further improvements.